In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas.io import sql
from sqlalchemy import create_engine
from matplotlib.pyplot import savefig

from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

engine = create_engine('')
cnn = engine.connect()

data = sql.read_sql("", cnn)
data.columns = ['idx','Location','Date','SO2','CO','O3','NO2','PM10','PM25']
data = pd.DataFrame(data,columns=['Location','Date','PM10'])
data_seoul=data[data.Location.isin(['서울'])]
data_seoul=data_seoul.sort_values(['Date'], ascending=[True])
del data_seoul['Location']
data_seoul=data_seoul.set_index(['Date'])
data_seoul=pd.rolling_mean(data_seoul,30)
data_seoul=data_seoul[29:]

train, test = data_seoul['PM10'][0:3261], data_seoul['PM10'][3261:len(data_seoul)]
data_seoul.index=data_seoul.index.to_datetime()

history = [x for x in train]

In [ ]:
import warnings
warnings.filterwarnings("ignore")

predictions = list()
%timeit
for i in range(len(test)):
    model = ARIMA(history, order=(1,1,1))
    fitted_model = model.fit(disp=0, method='css-mle')
    output = fitted_model.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[i]
    history.append(obs)
    
error = mean_squared_error(test, predictions)
print("MSE: %.3f"%error)

import math
rmserror = math.sqrt(error)
print("RMSE: %.3f"%rmserror)

In [ ]:
test_df=pd.DataFrame(test)
predict_df=pd.DataFrame(predictions)
predict_df.index=test_df.index
result=test_df.join(predict_df)
result.columns=['Observed','Predict']

In [ ]:
%matplotlib inline
result.plot(figsize=(16,10))